# Curso Transfer Learning

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

## Creando nuestro propio Clasificador de Reseñas 📦 🔍

En este lab aprenderás:

* [Transformers](https://huggingface.co/docs/transformers/v4.17.0/en/index)
* Descargar un dataset, prepararlo, entrenarlo, realizar finetuning y guardarlo.
* Manejo de plataformas como [Hugging Face Hub](https://huggingface.co/) y [Weights & Biases](https://wandb.ai/).

### 1) Descarga del dataset 🤓

Utilizaremos un conjunto de reseña de productos de Amazon [Amazon Reviews Dataset](https://www.kaggle.com/datasets/mexwell/amazon-reviews-multi).

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=dfeee9b9419babc416070998bb821e1081b52a3776a4815a2205762d4f306511
  Stored in directory: /root/.cache/pip/wheels/9f/af/22/bf406f913dc7506a485e60dce8143741abd0a92a19337d83a3
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.17
    Uninstalling kaggle-1.6.17:
      Successfully uninstalled kaggle-1.6.17


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"carlosbustillo","key":"60aea5be8b60f01d6ee908af46ecd025"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s amazon-reviews-multi

ref                                                       title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
mexwell/amazon-reviews-multi                              ✏️ Amazon Reviews Multi                             131MB  2023-07-25 11:35:14           1428         14  1.0              
karkavelrajaj/amazon-sales-dataset                        Amazon Sales Dataset                                  2MB  2023-01-17 06:21:15         129979        791  1.0              
jeromeblanchet/multidomain-sentiment-analysis-dataset     Johns Hopkins Multi-Domain Sentiment Dataset ∑∞       2GB  2020-01-14 15:36:51            382          9  0.5625           
dataup1/ogbn-products                                     OGBN-Products (Processed for PyG

In [ ]:
!kaggle datasets download -d mexwell/amazon-reviews-multi

Dataset URL: https://www.kaggle.com/datasets/mexwell/amazon-reviews-multi
License(s): other
100% 131M/131M [00:03<00:00, 60.2MB/s]
100% 131M/131M [00:03<00:00, 41.7MB/s]


In [ ]:
!unzip '/content/amazon-reviews-multi.zip'

Archive:  /content/amazon-reviews-multi.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: validation.csv          


### 2) Preparación de la data 👌

#### 2.1) Instalamos las dependencias 🙌

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
import pandas as pd
import numpy as np

import evaluate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from datasets import Dataset, DatasetDict

#### 2.2) Explorar dataset 👀

In [ ]:
df_train = pd.read_csv('/content/train.csv', sep=",", index_col=0)
df_test = pd.read_csv('/content/test.csv', sep=",", index_col=0)
df_val = pd.read_csv('/content/validation.csv', sep=",", index_col=0)

In [ ]:
df_train.head(3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,de_0203609,product_de_0865382,reviewer_de_0267719,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,de_0559494,product_de_0678997,reviewer_de_0783625,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement
2,de_0238777,product_de_0372235,reviewer_de_0911426,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore


Vamos a quedarnos exclusivamente con las reseñas en español, es decir `language = es`.

In [ ]:
df_train_es = df_train[df_train['language'] == 'es']
df_test_es = df_test[df_test['language'] == 'es']
df_val_es = df_val[df_val['language'] == 'es']

In [ ]:
df_train_es.head(3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
400000,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
400001,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
400002,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore


Reseteamos el índice

In [ ]:
df_train_es = df_train_es.reset_index(drop=True)
df_test_es = df_test_es.reset_index(drop=True)
df_val_es = df_val_es.reset_index(drop=True)

In [ ]:
df_train_es.head(3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore


Exploramos la distribución de los productos y estrellas

In [ ]:
df_train_es["product_category"].value_counts()

,count
product_category,
home,26962
wireless,25886
toy,13647
sports,13189
pc,11191
home_improvement,10879
electronics,10385
beauty,7337
automotive,7143


In [ ]:
df_train_es["stars"].value_counts()

,count
stars,
1,40000
2,40000
3,40000
4,40000
5,40000


#### 2.2) Acondicionar columnas 🔍

Vamos a asignar una etiqueta basado en la cantidad de estrellas:
- Si el número de estrellas es mayor o igual a 3, le asignamos una buena calificación (valor 1).
- Caso contrario es una mala calificación (valor 0).

In [ ]:
df_train_es['labels'] = df_train_es['stars'].apply(lambda x: 1 if x >= 3 else 0)
df_test_es['labels'] = df_test_es['stars'].apply(lambda x: 1 if x >= 3 else 0)
df_val_es['labels'] = df_val_es['stars'].apply(lambda x: 1 if x >= 3 else 0)

In [ ]:
df_train_es.head(2)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,labels
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics,0
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics,0


In [ ]:
df_train_es.tail(2)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,labels
199998,es_0930141,product_es_0788855,reviewer_es_0694290,5,Perfecto para el cumple de mi hijo,Recomendado,es,toy,1
199999,es_0859809,product_es_0378794,reviewer_es_0141717,5,"Súper bien! Las brochas son buenas, no sé meno...",Preciosas,es,home,1


#### 2.3) Adaptar formato del dataset 🔧

In [ ]:
# Convertir los DataFrames en objetos Dataset de la librería datasets
train_dataset = Dataset.from_pandas(df_train_es)
test_dataset = Dataset.from_pandas(df_test_es)
val_dataset = Dataset.from_pandas(df_val_es)

# Crear un DatasetDict con los conjuntos de datos
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Ver la estructura
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels'],
        num_rows: 5000
    })
})


### 3) Tokenización 📊

In [ ]:
model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Podemos visualizar el número total de tokens únicos que el tokenizador puede reconocer y procesar.

El vocabulario incluye todas las palabras, subpalabras y símbolos que el modelo ha aprendido a reconocer.

In [ ]:
tokenizer.vocab_size

50262

In [ ]:
text = "¡Aprendiendo sobre Transfer Learning en Platzi!"
tokenized_text = tokenizer.encode(text)

for token in tokenized_text:
    print(token, tokenizer.decode([token]))

0 <s>
1465 ¡
13327 Aprend
772 iendo
588  sobre
5269  Trans
1238 fer
1165  Le
25150 arning
334  en
2447  Pla
36422 tzi
55 !
2 </s>


In [ ]:
encoded_text = tokenizer(text, return_tensors="pt")
print(encoded_text)

{'input_ids': tensor([[    0,  1465, 13327,   772,   588,  5269,  1238,  1165, 25150,   334,
          2447, 36422,    55,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True)

In [ ]:
columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
print(encoded_dataset)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})


### 4) Finetuning de la convnet 😨

In [ ]:
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Input IDs -> hidden states -> outputs**

Resumen:
- Input IDs son la representación numérica del texto.
- El modelo recibe estos input IDs y calcula los hidden states, que son las representaciones internas del texto procesado.
- outputs almacena los hidden states y otras salidas del modelo.

In [ ]:
outputs = model(**encoded_text)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0219, -0.0384]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


**Vamos a evaluar el accuracy como métrica de rendimiento**

In [ ]:
metric = evaluate.load("accuracy")
print(metric)

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels)

### 5) Hugging Face Hub 🤗

El modelo entrenado lo vamos a subir a Hugging Face Hub así lo podemos compartir con el mundo 😎

**Importante:** La nueva credencial que vamos a crear debe tener permisos de escritura (write).

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `proyecto-platzi` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [ ]:
!git config --global credential.helper store

### 6) Entrenamiento 💪

In [ ]:
model_name = model_checkpoint.split("/")[-1]

In [ ]:
print(model_name)

roberta-base-bne


In [ ]:
batch_size = 8
num_train_epochs=2
num_train_samples = 20_000
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

In [ ]:
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    hub_model_id=f"{model_name}-platzi-project-nlp"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"],
    processing_class=tokenizer,
)

**Para este paso es necesario crearse una cuenta gratuita en [Weights & Biases](https://wandb.ai/home), porque el entrenamiento y las métricas se harán ahí.**

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.360500,0.381132,0.848800
2,0.256100,0.467120,0.854600


TrainOutput(global_step=5000, training_loss=0.3246493255615234, metrics={'train_runtime': 959.1194, 'train_samples_per_second': 41.705, 'train_steps_per_second': 5.213, 'total_flos': 1730826744362880.0, 'train_loss': 0.3246493255615234, 'epoch': 2.0})

### 7) Guardar el modelo 💾

Para eso vamos a hacer un push a Hugging Face Hub

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1736914196.09aa6524aa6c.2160.0:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cabustillo13/roberta-base-bne-platzi-project-nlp/commit/d1d1f304cf5f207e39eb89694ae54b895a87f727', commit_message='End of training', commit_description='', oid='d1d1f304cf5f207e39eb89694ae54b895a87f727', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cabustillo13/roberta-base-bne-platzi-project-nlp', endpoint='https://huggingface.co', repo_type='model', repo_id='cabustillo13/roberta-base-bne-platzi-project-nlp'), pr_revision=None, pr_num=None)

### 8) Hacer Predicciones en Producción 🤙

In [ ]:
from transformers import pipeline

Cargar el modelo una vez (al inicio de la aplicación)

In [ ]:
model_checkpoint = "cabustillo13/roberta-base-bne-platzi-project-nlp"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

config.json:   0%|          | 0.00/783 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Device set to use cuda:0


Ejemplos de uso

In [ ]:
pipe("me encanto el pantalon!!!")

[{'label': 'LABEL_1', 'score': 0.9975569248199463}]

In [ ]:
pipe("Te obligan a comprar dos unidades")

[{'label': 'LABEL_0', 'score': 0.9499565958976746}]

In [ ]:
pipe("la peor compra de mi vida!!! no recomiendo!")

[{'label': 'LABEL_0', 'score': 0.9918383955955505}]

### 9) Conclusiones

- Aprender sobre los distintos objetos y métodos que nos ofrece Transformers.

- Realizar el proceso completo de clasificación de reseñas con Transformers.

- Aprender tips sobre implementación con el uso de la GPU, Hugging Face Hub y Weights & Biases.

<br>
<br>
<br>

---

<br>
<br>


<img src="https://static.platzi.com/media/avatars/platziteam_8cfe6fc7-1246-4c9a-9f5d-d10d467443ee.png" width="100px">

[Platzi](https://platzi.com/) 🚀
